# Binance Profit Calculator 
`Written by @cryptoDey94 (telegram)` <br/>
Donations Appreciated :)<br/>
BTC Address: 1MLadoaCujyjXZUfbKR4greXiR84SaH4T8 <br/>

## Designed for use with [Gunbot](https://github.com/GuntharDeNiro/BTCT/wiki), but will work for any binance trading history. 


# Usage Instructions


## Download trading history from Binance

Log on to Binance and navigate to `Orders` -> `Trade History`. On the top right should be an option `Export complete trade history`. Click on that, chose a time period, and download the excel file. Make sure this .ipynb fle is in the same folder as `TradeHistory.xlsx`. (If filename is different than that, change the filename in the below cell). No personal information is contained in the trading history excel file.

In [1]:
filename = 'TradeHistory.xlsx'

## Run this file

#### Traditional Way
To run this file, I reccomend using Anaconda Navigator. To download, click the link below. Then open Anaconda Navigator, and click `Launch` under `Jupyter notebook`. Your browser will open with a screen to select which .ipynb file to open. Download and open this one, but before running, finish reading instructions. 
https://www.anaconda.com/download/

#### Easy way (Recommended)
Create an empty github repository for free, and just upload your `TradingHistory.xlsx` file and this .ipynb file to your github repo. Since github will only run this once, to update the results, simply change the file name and re upload it to github. Then github will detect it as a new file and re calculate. :)

## Notes on the program

The program calculates profits based on round-trip trades in the given time period. That is, for each coin pair, any sells that come before any buys and any buys that come after any sells will not be calculated. Thus, only full trades (trades that contain a buy and a sell) will be calcuated for profit. This is the most accurate way to determine how much profit you accumulated in the given time period.

The program calculates fees based on BNB. If you do not use BNB for fees, A. You should since its half the cost and B. you need to set the variable `use_fees` below to `False`. Note, this will calculate total revenue instead of total profit. 

In [2]:
use_fees = True

In [3]:
import pandas as pd
import numpy as np
import copy
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# initialize variables we will need
i = 0
totalProfit = 0.0
rawGroupDfs = {}
brokenDf = pd.DataFrame(columns=['Name', 'Total Bought', 'Total Sold', 'Profit', 'Fee'])
df = pd.read_excel(filename)
end = pd.to_datetime(df['Date'].iloc[0]).strftime('%I:%M %p on %B %d, %Y')
start = pd.to_datetime(df['Date'].iloc[-1]).strftime('%I:%M %p on %B %d, %Y')
print('\n')
print('Will calculate profit from ' + start + ' to ' + end)
print('\n')
print('Successfully Imported Excel File. Printing first 10 rows...')
df.head(10)



Will calculate profit from 03:53 PM on February 24, 2018 to 12:56 AM on March 02, 2018


Successfully Imported Excel File. Printing first 10 rows...


,Date,Market,Type,Price,Amount,Total,Fee,Fee Coin
0,2018-03-02 00:56:15,XLMBTC,BUY,0.000030,657.00,0.020032,0.010559,BNB
1,2018-03-02 00:56:15,XLMBTC,BUY,0.000030,497.00,0.015154,0.007987,BNB
2,2018-03-02 00:36:43,LTCBTC,BUY,0.019083,1.35,0.025762,0.013636,BNB
3,2018-03-02 00:36:43,LTCBTC,BUY,0.019079,0.45,0.008586,0.004545,BNB
4,2018-03-01 23:43:43,BTGBTC,BUY,0.010334,4.00,0.041336,0.021525,BNB
5,2018-03-01 23:26:16,ADABTC,BUY,0.000027,351.00,0.009610,0.005040,BNB
6,2018-03-01 19:33:51,HSRBTC,SELL,0.000868,29.15,0.025302,0.013086,BNB
7,2018-03-01 19:01:29,WTCBTC,BUY,0.001724,11.45,0.019744,0.010178,BNB
8,2018-03-01 18:43:03,RDNBTC,BUY,0.000252,121.00,0.030550,0.015441,BNB
9,2018-03-01 18:37:13,QTUMBTC,BUY,0.002412,16.05,0.038713,0.019871,BNB


The below code sanitizes the data and calculates profits based on the complete trades rules we have set above in `O(n)` time. 

In [4]:
xls = df.groupby(df['Market'])

#loop start - long cell bc all work being calculated in one go AKA O(n)
for name, group in xls:
    
    group = group.sort_values(by=['Date'], ascending=True)
    rawGroupDfs[name] = copy.deepcopy(group)
    
    # Create group dataframes for profit calculations
    group_df = pd.DataFrame(columns=['Total Bought', 'Total Sold', 'Profit', 'Fee'])

    # Remove sells from top
    group = group.reset_index()
    for index, row in group.iterrows():
        if row['Type'] == 'SELL':
            group.drop(index, inplace=True)
        else:
            break

    # Remove buys from bottom
    while group['Type'].count() > 0 and group.iloc[-1]['Type'] == 'BUY':
        group = group[:-1]


    # check for groups that only sold or bought, dont count them
    if len(group['Type'].unique()) == 1:
        continue
        
    
    # Calculate profits
    
    totalCostForCurrentGroup = 0.0
    totalRevenueForCurrentGroup = 0.0
    totalFeeForCurrentGroup = 0.0
    totalAmountForBuys = 0.0
    totalAmountForSells = 0.0
    totalProfitForCurrentGroup = 0.0
    for index, row in group.iterrows():
        
        if row['Type'] == 'BUY':
            totalAmountForBuys = totalAmountForBuys + float(row['Amount'])
            totalCostForCurrentGroup = totalCostForCurrentGroup + float(row['Total'])
            
        if row['Type'] == 'SELL':
            totalAmountForSells = totalAmountForSells + float(row['Amount'])
            totalRevenueForCurrentGroup = totalRevenueForCurrentGroup + float(row['Total'])
            
        totalFeeForCurrentGroup = totalFeeForCurrentGroup + float(row['Fee'])
        totalProfitForCurrentGroup = totalRevenueForCurrentGroup - totalCostForCurrentGroup

    # sometimes there is a slight difference in the amount bought / sold 
    # (maybe limit orders didnt get fully filled in time or owned coins from prev trades)
    # We will recalculate profits of those groups to be conservative to the lower amount
    
    totalProfitForCurrentGroup = totalRevenueForCurrentGroup - totalCostForCurrentGroup
    group_df = pd.DataFrame()
    
    if totalAmountForSells > totalAmountForBuys:
        group_df = group_df.set_value(i, 'Only counting', totalAmountForSells if totalAmountForBuys > totalAmountForSells else totalAmountForBuys)
        totalProfitForCurrentGroup = totalProfitForCurrentGroup * (totalAmountForBuys / totalAmountForSells)
        print('Fixing relative profit for group ' + name)
        print('\n')
        
    if totalAmountForSells < totalAmountForBuys:
        group_df = group_df.set_value(i, 'Only counting', totalAmountForSells if totalAmountForBuys > totalAmountForSells else totalAmountForBuys)        
        totalProfitForCurrentGroup = totalProfitForCurrentGroup * (totalAmountForSells / totalAmountForBuys)
        print('Fixing relative profit for group ' + name)
        print('\n')

    group_df = group_df.set_value(i, 'Total Bought', totalAmountForBuys)
    group_df = group_df.set_value(i, 'Total Sold', totalAmountForSells)
    group_df = group_df.set_value(i, 'Profit', totalProfitForCurrentGroup)
    group_df = group_df.set_value(i, 'Fee', totalFeeForCurrentGroup)
    group_df = group_df.set_value(i, 'Name', name)
    
    if totalProfitForCurrentGroup > 0 :
        resultDf = resultDf.append(group_df)
    else: 
        brokenDf = brokenDf.append(group_df)
        if totalProfitForCurrentGroup == 0.0:
            print('Group ' + name + ' had no round trip trades in this time period')
            print('\n')
        else:
            print('Problems calculating group ' + name + 
                  '. Total profit not added to final result is: ' + str(totalProfitForCurrentGroup))
            print('\n')

    i = i + 1
    
#loop over



Problems calculating group ADABTC. Total profit not added to final result is: -0.0006829199999999966


Problems calculating group BCDBTC. Total profit not added to final result is: -0.004281868999999994


Group BNBBTC had no round trip trades in this time period


Problems calculating group BTGBTC. Total profit not added to final result is: -0.002287919999999999


Group BTSBTC had no round trip trades in this time period


Problems calculating group CNDBTC. Total profit not added to final result is: -0.0012554500000000017


Group EOSBTC had no round trip trades in this time period


Problems calculating group ETCBTC. Total profit not added to final result is: -0.0020600600000000024


Problems calculating group ETHBTC. Total profit not added to final result is: -0.0008485330000000124


Problems calculating group HSRBTC. Total profit not added to final result is: -0.0011878700000000075


Group ICXBNB had no round trip trades in this time period


Group IOTABTC had no round trip trades in

In [8]:
from IPython.display import display

for raw in rawGroupDfs:
    display(rawGroupDfs[raw])


,Date,Market,Type,Price,Amount,Total,Fee,Fee Coin
191,2018-02-24 15:53:28,ADABTC,BUY,0.000031,315.0,0.009689,0.005191,BNB
46,2018-02-28 13:20:31,ADABTC,SELL,0.000029,315.0,0.009223,0.004629,BNB
28,2018-03-01 01:36:22,ADABTC,BUY,0.000029,344.0,0.009845,0.004886,BNB
10,2018-03-01 14:34:14,ADABTC,SELL,0.000028,344.0,0.009629,0.004946,BNB
5,2018-03-01 23:26:16,ADABTC,BUY,0.000027,351.0,0.009610,0.005040,BNB


,Date,Market,Type,Price,Amount,Total,Fee,Fee Coin
164,2018-02-26 04:51:37,BCDBTC,SELL,0.004182,9.246,0.038667,0.019864,BNB
147,2018-02-26 05:01:01,BCDBTC,BUY,0.004046,9.719,0.039323,0.020199,BNB
144,2018-02-26 05:06:19,BCDBTC,SELL,0.004050,9.719,0.039362,0.020272,BNB
137,2018-02-26 07:25:38,BCDBTC,BUY,0.003931,9.746,0.038312,0.019532,BNB
120,2018-02-26 20:10:33,BCDBTC,SELL,0.003632,9.746,0.035397,0.018418,BNB
99,2018-02-27 04:48:30,BCDBTC,BUY,0.003548,9.837,0.034902,0.018026,BNB
84,2018-02-27 19:21:03,BCDBTC,SELL,0.003405,9.837,0.033495,0.016612,BNB
41,2018-02-28 16:19:33,BCDBTC,BUY,0.003260,10.139,0.033053,0.016400,BNB


,Date,Market,Type,Price,Amount,Total,Fee,Fee Coin
72,2018-02-28 05:07:27,BNBBTC,BUY,0.001014,5.0,0.005069,0.0025,BNB


,Date,Market,Type,Price,Amount,Total,Fee,Fee Coin
189,2018-02-25 00:36:18,BTGBTC,SELL,0.011731,3.66,0.042935,0.022748,BNB
169,2018-02-25 14:27:21,BTGBTC,BUY,0.011445,3.75,0.042919,0.022395,BNB
106,2018-02-27 00:54:34,BTGBTC,SELL,0.011143,3.75,0.041786,0.021532,BNB
73,2018-02-28 04:21:51,BTGBTC,BUY,0.010907,3.93,0.042865,0.021304,BNB
42,2018-02-28 16:08:16,BTGBTC,SELL,0.010613,2.63,0.027912,0.013816,BNB
43,2018-02-28 16:08:16,BTGBTC,SELL,0.010613,1.30,0.013797,0.006834,BNB
4,2018-03-01 23:43:43,BTGBTC,BUY,0.010334,4.00,0.041336,0.021525,BNB


,Date,Market,Type,Price,Amount,Total,Fee,Fee Coin
187,2018-02-25 00:36:28,BTSBTC,SELL,0.000021,468.0,0.010001,0.005297,BNB
188,2018-02-25 00:36:28,BTSBTC,SELL,0.000021,1184.0,0.025302,0.013400,BNB
65,2018-02-28 05:44:54,BTSBTC,BUY,0.000020,1230.0,0.025006,0.012404,BNB


...

,Date,Market,Type,Price,Amount,Total,Fee,Fee Coin
153,2018-02-26 04:55:32,XLMBTC,SELL,0.000036,873.0,0.031149,0.016009,BNB
121,2018-02-26 19:08:36,XLMBTC,BUY,0.000035,1134.0,0.039633,0.020670,BNB
86,2018-02-27 18:04:11,XLMBTC,SELL,0.000033,474.0,0.015822,0.007783,BNB
87,2018-02-27 18:04:11,XLMBTC,SELL,0.000033,660.0,0.022031,0.010833,BNB
67,2018-02-28 05:25:53,XLMBTC,BUY,0.000033,1139.0,0.037211,0.018408,BNB
14,2018-03-01 10:20:59,XLMBTC,SELL,0.000031,239.0,0.007442,0.003742,BNB
15,2018-03-01 10:20:59,XLMBTC,SELL,0.000031,900.0,0.028035,0.014102,BNB
0,2018-03-02 00:56:15,XLMBTC,BUY,0.000030,657.0,0.020032,0.010559,BNB
1,2018-03-02 00:56:15,XLMBTC,BUY,0.000030,497.0,0.015154,0.007987,BNB


,Date,Market,Type,Price,Amount,Total,Fee,Fee Coin
152,2018-02-26 04:55:48,XMRBTC,BUY,0.028408,0.310,0.008806,0.004528,BNB
151,2018-02-26 04:55:54,XMRBTC,BUY,0.028408,0.315,0.008949,0.004600,BNB
149,2018-02-26 04:56:04,XMRBTC,BUY,0.028408,0.079,0.002244,0.001154,BNB


,Date,Market,Type,Price,Amount,Total,Fee,Fee Coin
150,2018-02-26 04:55:55,XRPBTC,SELL,0.000093,286.0,0.026541,0.013655,BNB
122,2018-02-26 16:15:14,XRPBTC,BUY,0.000091,157.0,0.014243,0.007394,BNB
123,2018-02-26 16:15:14,XRPBTC,BUY,0.000091,231.0,0.020959,0.010880,BNB
88,2018-02-27 17:30:46,XRPBTC,SELL,0.000088,388.0,0.034020,0.017012,BNB
68,2018-02-28 05:16:05,XRPBTC,BUY,0.000086,389.0,0.033357,0.016521,BNB


,Date,Market,Type,Price,Amount,Total,Fee,Fee Coin
174,2018-02-25 00:39:53,XVGBTC,SELL,0.000006,4878.0,0.028731,0.015225,BNB
57,2018-02-28 06:38:43,XVGBTC,BUY,0.000006,757.0,0.004171,0.002082,BNB
56,2018-02-28 06:38:44,XVGBTC,BUY,0.000006,1000.0,0.005510,0.002750,BNB
55,2018-02-28 06:39:02,XVGBTC,BUY,0.000006,1499.0,0.008259,0.004126,BNB
54,2018-02-28 06:39:47,XVGBTC,BUY,0.000006,3096.0,0.017059,0.008539,BNB
35,2018-02-28 18:50:52,XVGBTC,SELL,0.000005,4377.0,0.023154,0.011619,BNB
36,2018-02-28 18:50:52,XVGBTC,SELL,0.000005,1975.0,0.010448,0.005242,BNB
32,2018-03-01 00:23:12,XVGBTC,BUY,0.000005,1257.0,0.006486,0.003185,BNB
33,2018-03-01 00:23:12,XVGBTC,BUY,0.000005,5472.0,0.028290,0.013867,BNB


,Date,Market,Type,Price,Amount,Total,Fee,Fee Coin
142,2018-02-26 05:16:54,ZECBTC,SELL,0.040703,0.178,0.007245,0.003718,BNB
143,2018-02-26 05:16:54,ZECBTC,SELL,0.040703,0.456,0.018561,0.009532,BNB
125,2018-02-26 14:12:31,ZECBTC,BUY,0.039842,0.886,0.035300,0.018132,BNB
95,2018-02-27 08:50:39,ZECBTC,SELL,0.039001,0.886,0.034555,0.016783,BNB
80,2018-02-27 23:41:43,ZECBTC,BUY,0.038204,0.892,0.034078,0.017020,BNB
45,2018-02-28 15:15:20,ZECBTC,SELL,0.036890,0.892,0.032906,0.016161,BNB


In [ ]:
print('Successful calculation of below groups: If -, then that means we are counting all')
resultDf.fillna('-')

In [ ]:
profitDf = pd.DataFrame(columns=['Total Profit BTC', 'Total Fees BNB'])
profitDf['Total Profit BTC'] = [resultDf['Profit'].sum()]
profitDf['Total Fees BNB'] = [resultDf['Fee'].sum()]
profitDf.index = ['Total']

In [ ]:
print('These were not added to final result. If all 0, then no round trip trades for these groups exist in the given time period. If running many pairs, this is expected.')
brokenDf

In [ ]:
#USD Profits
usProfitDf = pd.DataFrame(index=[8000, 10000, 12000, 15000, 20000, 25000, 50000, 100000], 
                          columns=['USD Profit if cost of BTC is ...'])
usProfitDf['USD Profit if cost of BTC is ...'] = usProfitDf.index * resultDf['Profit'].sum()
usProfitDf.index.name = 'Price BTC'

In [ ]:
# Get profit in USD from real time BTC price from coinmarketcap

import requests

response = requests.get('https://api.coinmarketcap.com/v1/ticker/bitcoin/') 
response2 = requests.get('https://api.coinmarketcap.com/v1/ticker/binance-coin/') 
realTimeProfitDf = pd.DataFrame(index=['Total'])
if use_fees:
    realTimeProfitDf['USD Profit if cost of BTC is ...'] = float(response.json()[0]['price_usd']) * resultDf['Profit'].sum() - float(response2.json()[0]['price_usd']) * resultDf['Fee'].sum()
else:
    realTimeProfitDf['USD Profit if cost of BTC is ...'] = float(response.json()[0]['price_usd']) * resultDf['Profit'].sum()

In [ ]:
profitDf

In [ ]:
pd.options.display.float_format = '${:,.2f}'.format
print(usProfitDf.to_string(index=True))

In [ ]:
resultDf['USD Profit'] = (float(response.json()[0]['price_usd']) * resultDf['Profit'])
resultDf = resultDf.sort_values(by='USD Profit', ascending=False)
ax = resultDf.plot(x='Name', y='USD Profit', kind='bar', figsize=(16,7), title='Proft in USD per pair')
plt.show()
finalDf = pd.DataFrame(resultDf[['Name','USD Profit']], index=None)

print(finalDf.to_string(index=False))

In [ ]:
import datetime as datetime
print('\n')
print ('Current Bitcoin Price as of ' + datetime.datetime.now().strftime('%I:%M %p on %B %d, %Y') + ' is: ' + "${:,.2f}".format(float(response.json()[0]['price_usd'])))
print('\n')

In [ ]:
feeCost = float(response2.json()[0]['price_usd']) * float(resultDf['Fee'].sum())
label = str('Profit from ' + str(start) + ' to ' + str(end) + ' in USD')
realTimeProfitDf.columns=[label]
print('\n')
print(str('Fee\'s from BNB ' + str(start) + ' to ' + str(end) + ' currently costs ' + "${:,.2f}".format(feeCost) + ' in USD.'))
print('\n')
realTimeProfitDf

In [ ]:
end = pd.to_datetime(df['Date'].iloc[0])
start = pd.to_datetime(df['Date'].iloc[-1])

diff = end - start

days, seconds = diff.days, diff.seconds
if days == 0:
    days = 1
hours = days * 24.0 + seconds // 3600.0
minutes = hours * 60.0

priceBTC = float(response.json()[0]['price_usd'])
avgProfMin = resultDf['Profit'].sum() * priceBTC / minutes
avgProfHour = resultDf['Profit'].sum() * priceBTC / hours
avgProfDay = resultDf['Profit'].sum() * (priceBTC / days)

btcProfit = resultDf['Profit'].sum()
print('Average profit per minute is ' "${:,.2f}".format(avgProfMin) + ' or ' + "{:,.15f}".format(btcProfit/minutes) + ' BTC.')
print('Average profit per hour is ' "${:,.2f}".format(avgProfHour) + ' or ' + "{:,.15f}".format(btcProfit/hours) + ' BTC.')
print('Average daily profit is ' "${:,.2f}".format(avgProfDay) + ' or ' + "{:,.15f}".format(btcProfit/days) + ' BTC.')

print('Not bad for a bot trading for you!')